# VENDING MACHINE

## Welcome to my Vending machine!

Please feel free to use it by selecting **Run All** :)

In [60]:
# Generate data structures

# TODO: MAKE THIS READ FROM FILE INPUT 
productDict = {"A1": {"Coffee": 0}, "A2": {"Tea": 5}, "B1": {"Apple Juice": 10}, "B2": {"Orange Juice": 10}}
extraIngredients = {"Sugar": 20, "Ice": 10}

In [61]:
# Function to display nested dictionary of products and keys

def displayNestedDict():
    """
    Displays Product list and if product is unavailable.
    """
    for product_ID, product_name in productDict.items(): 
        for key in product_name:
            if product_name[key] == 0: # Message if out of stock
                message = "NOT AVAILABLE"
            else:
                message = " "
            print(product_ID + " " + key + " " + message)

In [62]:
# Function to accept string input from user
choice = input("Please enter your selection: ").upper()

In [ ]:
# Main Program
